In [ ]:
import os
import zipfile
import pandas as pd

In [ ]:
print(os.listdir("../input"))


In [ ]:
with zipfile.ZipFile('../input/dogs-vs-cats/train.zip', 'r') as zip_ref:
    zip_ref.extractall('/tmp/dogs-vs-cats/train')

In [ ]:
with zipfile.ZipFile('../input/dogs-vs-cats/test1.zip', 'r') as zip_ref:
    zip_ref.extractall('/tmp/dogs-vs-cats/test')

In [ ]:
base_dir = "/tmp/dogs-vs-cats"
train_dir = os.path.join(base_dir, 'train/train')
test_dir = os.path.join(base_dir, 'test/test')


In [ ]:
filenames = os.listdir("/tmp/dogs-vs-cats/train/train")

In [ ]:

categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append("dog")
    else:
        categories.append("cat")

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [ ]:
df.tail()

In [ ]:
from sklearn.model_selection import train_test_split
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Reshape,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
model = Sequential([
    Conv2D(filters=32,kernel_size=(3,3), input_shape = (64, 64, 3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(filters=32,kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    Conv2D(filters=64,kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.25),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [ ]:
rm -rf logs

In [ ]:
%load_ext tensorboard
log_folder = 'logs'

In [ ]:
callbacks = [
            # EarlyStopping(patience = 10,monitor="accuracy",mode="max"),
            TensorBoard(log_dir=log_folder)
    
            ]

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   shear_range=0.2,
                                   zoom_range=0.2, 
                                   horizontal_flip=True,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1
                                   )

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
test_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_dir

In [ ]:
training_set = train_datagen.flow_from_dataframe(
                                                train_df,
                                                train_dir,
                                                 seed=101,                                                 
                                                 target_size=(64, 64),
                                                 batch_size=32,
                                                 x_col='filename',
                                                 y_col='category',
                                                 class_mode='binary')

In [ ]:
validation_data = validation_datagen.flow_from_dataframe(
                                                validate_df,
                                                train_dir,
                                                 seed=101,                                                 
                                                 target_size=(64, 64),
                                                 batch_size=32,
                                                 x_col='filename',
                                                 y_col='category',
                                                 class_mode='binary')

In [ ]:
history = model.fit(
          training_set,
          epochs=25, 
          validation_data=validation_data,
          callbacks=callbacks)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
import pandas as pd
losses = pd.DataFrame(history.history)

In [ ]:
losses.head()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,6))
plt.xlabel('Epochs')
plt.ylabel('Loss')
losses['loss'].plot(label='Training Loss')
losses['val_loss'].plot(label='Validation Loss')
plt.legend(loc='center')

In [ ]:
import numpy as np
from keras.preprocessing import image

In [ ]:
test_filenames = os.listdir("/tmp/dogs-vs-cats/test/test1")
test_df = pd.DataFrame({
    'filename': test_filenames
})

In [ ]:
test_df.head()

In [ ]:
nb_samples = test_df.shape[0]


In [ ]:
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    "/tmp/dogs-vs-cats/test/test1", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=(64,64),
    batch_size=32,
    shuffle=False
)

In [ ]:
predictions = model.predict(test_generator)

In [ ]:
predictions

In [ ]:
categories = []
for prediction in predictions:
  if prediction >= 0.5:
    label=1
    categories.append(label)
  else:
    label=0
    categories.append(label)

In [ ]:
test_df.head()

In [ ]:

def get_filenames(filename):
  image = filename.split('.')[0]
  return image

In [ ]:
images = list(test_df["filename"].apply(get_filenames))

In [ ]:
submission_df = pd.DataFrame({
    'id': images,
    'label': categories
})

In [ ]:
submission_df.head()

In [ ]:
submission_df.to_csv("submission.csv",index=False)

In [ ]:
%tensorboard --logdir={log_folder}
# %tensorboard line magic to start